In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


def scale_axis(data: pd.DataFrame, scale:dict) -> pd.DataFrame:
    data = data.copy()
    new_list = []
    for key in data["name"].to_list():
        new_list.append(scale[key])
    data["scale"] = new_list
    data["x"] = data["x"] / data["scale"]
    data["y"] = data["y"] / data["scale"]
    data["z"] = data["z"] / data["scale"]
    return data

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "next",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
            }

# def frame_args_hidden(frames, period):
    

In [ ]:
planets_data = pd.read_csv("planets_data.csv", index_col=0)
planets_data = planets_data[planets_data.name != "Venus"]
number_of_planets = planets_data.shape[0]
planets_data

In [ ]:
import datetime

steps = 30
scale = {"Sun": 1*10e6, "Mercury": 1*10e7, "Venus":1*10e7, "Earth":1*10e7, "Mars":1*10e7,
         "Jupiter":3*10e7, "Saturn":1*10e7, "Uranus":1*10e7, "Neptune": 1*10e7}
scale = {"Sun": 1, "Mercury": 1, "Venus":1, "Earth":1, "Mars":1,
         "Jupiter":1, "Saturn":1, "Uranus":1, "Neptune": 1}

position_history = pd.read_csv("position_history.csv", index_col=0)
position_history["time"] = pd.to_datetime(position_history["time"]).astype("datetime64[D]")
# position_history = scale_axis(position_history, scale=scale)
position_history = position_history[position_history.name != "Venus"]
# position_history["radius"] = np.power(position_history["radius"], 1/4) * 10e6

position_history["radius"][position_history["name"] == "Sun"] = \
    position_history["radius"][position_history["name"] == "Sun"]/7000
position_history["radius"] = np.sqrt(position_history["radius"]) / 20

position_history = position_history.sort_values("time")
time_step = ((max(position_history["time"]) - min(position_history["time"])) / steps).days
position_history["time"] = position_history["time"].dt.strftime("%Y:%m:%d")

x_limit = position_history["x"].abs().max()
y_limit = position_history["y"].abs().max()
z_limit = position_history["z"].abs().max()

position_history = position_history.reset_index()
# position_history.head(number_of_planets)


downsampled_position_history = position_history[0:number_of_planets]
for index in range(number_of_planets*time_step, position_history.shape[0], number_of_planets*time_step):
    downsampled_position_history = downsampled_position_history.append(position_history[index:index+number_of_planets])

print(f"Data shape:{downsampled_position_history.shape}")
downsampled_position_history.head(number_of_planets)

In [ ]:
fig = px.scatter_3d(downsampled_position_history, x="x", y="y", z="z", color="name", size="radius",
                    animation_frame="index", animation_group="name", template="plotly_dark", opacity=0,
                    color_discrete_sequence=["orange", "crimson", "cornflowerblue", "cyan", 
                                             "khaki", "lightcyan", "chocolate", "green"],
                   )
# size="radius",

# scene = dict(xaxis = dict(range=[-x_limit, x_limit], autorange=False, visible=False),
#              yaxis = dict(range=[-y_limit, y_limit], autorange=False, visible=False),
#              zaxis = dict(range=[-z_limit, z_limit], autorange=False, visible=False), 
#              )

# updatemenus = [
#                 {
#                 "buttons": [
#                         {
#                         "args": [None, frame_args(30)],
#                         "label": "&#9654;", # play symbol
#                         "method": "animate",
#                         },
                    
#                         {
#                         "args": [[None], frame_args(0)],
#                         "label": "&#9724;", # pause symbol
#                         "method": "animate",
#                         },
#                     ],
#                 },
#             ]

# sliders = [
#             {
#             "steps": [
#                 {
#                     "args": [[frame.name], frame_args(0)],
#                     "label": str(index),
#                     "method": "animate",
#                 }
#                 for index, frame in enumerate(fig.frames)
#             ],
#             }
#         ]
               

# fig.update_layout(height=800 ,scene_aspectmode='cube', scene = scene, sliders=sliders, updatemenus=updatemenus)
fig

## Before saving clear simulation (eg. turn cell to markdown) or the file will take 100mb

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)]

In [ ]:
# color_sequnece=["orange", "crimson", "cornflowerblue", "cyan", "khaki", "lightcyan", "chocolate", "green"]
color_sequnece = ["orange", "#AA0DFE", "chocolate", "cyan", "cornflowerblue", "#B2828D", "crimson", "#19D3F3"]
fig = px.scatter_3d(position_history, x="x", y="y", z="z", color="name", size="radius",
                    animation_frame="index", animation_group="name", template="plotly_dark", opacity=0,
                    color_discrete_sequence=color_sequnece)

fig.add_traces(data=[go.Scatter3d(x=position_history["x"][position_history["name"] == name],
                                  y=position_history["y"][position_history["name"] == name],
                                  z=position_history["z"][position_history["name"] == name],
                                  mode="lines",
                                  showlegend=False,
                                  marker=dict(color="grey")
                                 )
                     for name in planets_data["name"].to_list()
                    ]
              )

updatemenus = [
                {
                "buttons": [
                        {
                        "args": [None, frame_args(30)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                        },
                    
                        {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                        },
                    ],
                },
            ]

sliders = [
            {
            "steps": [
                {
                    "args": [[frame.name], frame_args(0)],
                    "label": str(index),
                    "method": "animate",
                }
                for index, frame in enumerate(fig.frames)
            ],
            }
        ]

scene = dict(xaxis = dict(range=[-x_limit, x_limit], autorange=False, visible=False),
             yaxis = dict(range=[-y_limit, y_limit], autorange=False, visible=False),
             zaxis = dict(range=[-z_limit, z_limit], autorange=False, visible=False), 
             )
fig.update_layout(height=800, scene_aspectmode='cube', updatemenus=updatemenus,
                  sliders=sliders, scene=scene)
fig

In [ ]:
import plotly.graph_objects as go
x = np.arange(10)
y = np.arange(10)*-10
z = np.arange(10)*2
test = px.scatter_3d(x=x, y=y, z=z)
test.add_traces(data=[go.Scatter3d(x=x, y=y, z=z*K/10, mode="lines", showlegend=False) for K in range(10)])
test